In [1]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
"""
Date: 2023/12/11 22:40
Desc: 东方财富-ETF行情
https://quote.eastmoney.com/sh513500.html
"""
import pandas as pd

import requests

import datetime

from datasource import *

In [2]:
def agg_ticks_to_klines(source_df: pd.DataFrame, target_interval='5S'):
    """
    根据 Tick 数据生成 K 线数据
    :param tick_data: 实时 Tick 数据 (pandas DataFrame)
    :param interval: K 线的时间间隔，默认为 5S 钟
    :return: K 线数据 (pandas DataFrame)
    """

    # 确保'time'列是datetime类型
    source_df['datetime'] = pd.to_datetime(source_df['datetime'])

    # 将标的和时间作为分组关键字，使用groupby聚合
    grouped_df = source_df.groupby(['code', pd.Grouper(key='datetime', freq=target_interval)])

    # 对每个分组应用聚合操作，例如计算OHLC和总成交量
    kline_df = grouped_df.agg({
        'price': ['first', 'max', 'min', 'last'],  # open, high, low, close
        'volume': 'sum'
    })

    # 重置索引，将分组关键字还原为列
    kline_df = kline_df.reset_index()

    kline_df.columns = [
        "code",
        "date",
        "open",
        "high",
        "low",      
        "close",
        "volume",
    ]

    # 打印结果
    # print(kline_df)

    return kline_df

In [3]:
tick_data = fund_etf_spot_em()

In [13]:
def process_tick_and_k_line(tick_data):
    # 获取实时 Tick 数据
    # tick_data = fund_etf_spot_em()
    
    if '日期' in tick_data.columns:
        tick_data['datetime'] = pd.to_datetime(tick_data['日期'])
    else:
        if 'datetime' not in tick_data.columns:
            # 获取本地当前时间
            current_datetime = datetime.datetime.now()   
            # 在 Tick 数据中插入本地当前时间列到第一列
            tick_data['datetime'] = current_datetime
            tick_data['datetime'] = pd.to_datetime(tick_data['datetime'])
        else:
            tick_data['datetime'] = pd.to_datetime(tick_data['datetime'])

    # 将 "datetime" 列转换为 datetime 类型
    # tick_data['datetime'] = pd.to_datetime(tick_data['datetime'])

    # 将 "datetime" 列移到第一列
    tick_data = tick_data[['datetime'] + [col for col in tick_data.columns if col != 'datetime']]

    print(tick_data.columns)

    tick_data.columns = [
        "datetime",
        "code",
        "name",
        "price",
        "涨跌额",
        "涨跌幅",
        "volume",
        "成交额",
        "开盘价",
        "最高价",
        "最低价",
        "昨收",
        "换手率",
        "流通市值",
        "总市值",
    ]

    # 生成 K 线数据（默认：每 5 分钟生成一个 K 线）
    kline_data = agg_ticks_to_klines(tick_data)
    

In [16]:
tick_data

,代码,名称,最新价,涨跌额,涨跌幅,成交量,成交额,开盘价,最高价,最低价,昨收,换手率,流通市值,总市值,datetime
0,513000,日经225ETF易方达,1.358,0.042,3.19,615540.0,8.338735e+07,1.338,1.379,1.338,1.316,19.64,425612692,425612692,2024-01-10 15:11:15.155481
1,561910,电池ETF,0.460,0.009,2.00,581176.0,2.661113e+07,0.451,0.466,0.446,0.451,6.20,431515240,431515240,2024-01-10 15:11:15.155481
2,560980,光伏30ETF,0.575,0.011,1.95,902153.0,5.188544e+07,0.558,0.586,0.556,0.564,25.16,206150150,206150150,2024-01-10 15:11:15.155481
3,562880,电池ETF基金,0.486,0.009,1.89,199698.0,9.628481e+06,0.476,0.492,0.470,0.477,3.89,249269206,249269206,2024-01-10 15:11:15.155481
4,510630,消费30ETF,0.919,0.015,1.66,136781.0,1.250530e+07,0.904,0.923,0.898,0.904,4.45,282465222,282465222,2024-01-10 15:11:15.155481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,512980,传媒ETF,0.673,-0.019,-2.75,1189997.0,8.066521e+07,0.685,0.690,0.673,0.692,2.19,3649291567,3649291567,2024-01-10 15:11:15.155481
866,516770,游戏动漫ETF,0.956,-0.027,-2.75,169776.0,1.637628e+07,0.983,0.983,0.955,0.983,12.42,130635488,130635488,2024-01-10 15:11:15.155481
867,516620,影视ETF,0.888,-0.027,-2.95,168729.0,1.508668e+07,0.914,0.914,0.887,0.915,38.12,39309540,39309540,2024-01-10 15:11:15.155481
868,159805,传媒ETF,0.929,-0.029,-3.03,193836.0,1.816695e+07,0.954,0.954,0.929,0.958,10.51,171356280,171356280,2024-01-10 15:11:15.155481


In [14]:
kline_data = process_tick_and_k_line(tick_data)

Index(['datetime', '代码', '名称', '最新价', '涨跌额', '涨跌幅', '成交量', '成交额', '开盘价', '最高价',
       '最低价', '昨收', '换手率', '流通市值', '总市值'],
      dtype='object')
